In [2]:
#!/usr/bin/env python

# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Outlines document text given an image.
Example:
    python doctext.py resources/text_menu.jpg
"""
# [START full_tutorial]
# [START imports]
import argparse
from enum import Enum
import io

from google.cloud import vision
from google.cloud.vision import types
from PIL import Image, ImageDraw
from google.oauth2 import service_account
# [END imports]

credentials = service_account.Credentials. from_service_account_file('googleapikey.json')

class FeatureType(Enum):
    PAGE = 1
    BLOCK = 2
    PARA = 3
    WORD = 4
    SYMBOL = 5


def draw_boxes(image, bounds, color):
    """Draw a border around the image using the hints in the vector list."""
    # [START draw_blocks]
    draw = ImageDraw.Draw(image)

    for bound in bounds:
        draw.polygon([
            bound.vertices[0].x, bound.vertices[0].y,
            bound.vertices[1].x, bound.vertices[1].y,
            bound.vertices[2].x, bound.vertices[2].y,
            bound.vertices[3].x, bound.vertices[3].y], None, color)
    return image
    # [END draw_blocks]


def get_document_bounds(image_file, feature):
    # [START detect_bounds]
    """Returns document bounds given an image."""
    client = vision.ImageAnnotatorClient(credentials=credentials)

    bounds = []

    with io.open(image_file, 'rb') as image_file:
        content = image_file.read()

    image = types.Image(content=content)

    response = client.document_text_detection(image=image)
    document = response.full_text_annotation

    # Collect specified feature bounds by enumerating all document features
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    for symbol in word.symbols:
                        if (feature == FeatureType.SYMBOL):
                            bounds.append(symbol.bounding_box)

                    if (feature == FeatureType.WORD):
                        bounds.append(word.bounding_box)

                if (feature == FeatureType.PARA):
                    bounds.append(paragraph.bounding_box)

            if (feature == FeatureType.BLOCK):
                bounds.append(block.bounding_box)

        if (feature == FeatureType.PAGE):
            bounds.append(block.bounding_box)

    # The list `bounds` contains the coordinates of the bounding boxes.
    # [END detect_bounds]
    return bounds, document, response


def render_doc_text(filein, fileout):
    # [START render_doc_text]
    image = Image.open(filein)
    bounds, document1, response1 = get_document_bounds(filein, FeatureType.PAGE)
    draw_boxes(image, bounds, 'blue')
    bounds, document2, response2 = get_document_bounds(filein, FeatureType.PARA)
    draw_boxes(image, bounds, 'red')
    #print('Red Text')
    #print(document2.text)
    bounds, document3, response3 = get_document_bounds(filein, FeatureType.WORD)
    draw_boxes(image, bounds, 'yellow')
    #print('Detected Text')
    #print(document3.text)
    #print('RAW JSON')
    #print(document3)

    if fileout is not 0:
        image.save(fileout)
    else:
        image.show()
        
    return response3
    # [END render_doc_text]



In [3]:
#Read json from Google Vision response, convert to string, write to a file

def write_json(response, json_out):
    import json
    from google.protobuf.json_format import MessageToJson
    import os
    
    response = response

    #Google Vision files come in as a VisionImageAnnotation type.  MessageToJson converts data to str type

    json_data_1 = MessageToJson(response)
    #print(json_data_1.type)

    #Initialize Dict
    final_json = {}

    #convert str to a dict
    final_json = json.loads(json_data_1)

    response_dict = {}

    #Extract data within 'textAnnotations' value in previous dict to a new dict(or list)? where you can call response_dict['description'] to retreive all text values detected
    response_dict = final_json['textAnnotations']
    
    file_name = json_out
    
    #Initiate a list for loading dict data
    all_data = []

    #Append the dict loaded with json data to variable

    all_data.append(final_json['textAnnotations'])

    #Write the all_data data to json file of file _name
    with open(file_name, 'w') as f:
        json.dump(all_data, f)
        
    return final_json

In [9]:
def extract_words(final_json):
    textAnnotations = {}
    textAnnotations = final_json['textAnnotations']


    all_words = []
    n = len(textAnnotations)
    print(n)
    
    #each dict element in textAnnotations its in own dict.  Loop through entire textAnnotations to extract only the words
    #textAnnotations[0] is all of the words in the page
    #textAnnotations[1 and more] are the individual words

    for x in range(n):
        descriptions = {}
        descriptions = textAnnotations[x]
        all_words.append(descriptions['description'])
    print(all_words)

In [10]:
#Main code to execute tests

#May want to add code to iterate across all test images test_file_names = []

#Full path of image file
file_name = 'Images/02_Engine_Rod.jpg'
#Full path of output file with bounded boxes
file_out = 'Output/DELETE.jpg'
#Desired path of outfile json file of all results of google vision api
json_out = 'Output/02_Response.json'
# response contains the original document text
response = render_doc_text(file_name, file_out)

#write_json saves 
final_json = {}

final_json = write_json(response, json_out)
print(type(final_json))

extract_words(final_json)


<class 'dict'>
475
["45* SURFACE TO INTERSECT NORMAL\nSURFACE WITHIN 1.005 OF CENTERLINE\n0.410 - 420\nR06\n.225\n214.95 A REDRAWN WITHOUT\nCHANGE WAS\nWAN DYKE R3234\n7.395 BOD 145CHAMI\nADOGO, TWO\nPLACES R SS\n7-8-95 COM.3892-3907\nW.383.33 35230513\n7-8.95 CM 100208\n.485\n-485 449 4R06 I .225\nR.02-03 vit\n,475\nSECT L-L\nDOUBLE SIZE\n- THESE HOLES MUST BE PARALLEL WITHIN\n1.002 IN 3.00 INCHES IN BOTH PLANES\n35735\n45* +030\n0-5 L DM.501016\nWAS .593.610\n1:17.95 SECT RR. SECT SS.\n& VE WIN CIRCLE\nI REMOVED\n1705 G NOTE REVISED\nIR.06\n9988\n99983\n099830\n099855\nROD TO BE CORRECTED\nFOR WEIGHT BY\nMACHINING AND DRILLING\nTHIS LUG AS SHOWN\n1.422\nMIN\nTRAVEL OF CUTTER\nFOR MACHINING\n1.335 MIN\nFOR SERVICE\nONLY\n607\n62\n42.\nJopek\nTEX\nMEASURE FINISH BORE\nDIA IN THIS DIRECTION,\nMUST BE ROUND\nWITHIN .0005 TIR\n229 +0° 15' MIN\nCUTTER\nL225' 0'15' MAX\nCUTTER AT\n123\n02.37451\n02.3740\nR3.00\nCUTTER\n281\nK: IP20 Utosas\n00 MIN\nOi\ni\nM\n1482026\nMLMR100\nTWO\n2.0245 PLA

In [43]:
print(response_dict)

{'locale': 'en', 'description': '10\nITEM NUMBER\nREVISIONS\n3170649\n--\n353.7\n[13.93]\nZone e lasciato\n4 13\n4\nr2x Ø 12.7 1.5] X 25.4 [1]\nSLOT THRU\n+ 95.5\n- 106.9\n[4.21]\nDesca Ption\nIN\nporno Po\n00 OSC 8.3-HX\n63.5\n[2.50]\n(LA9104\nT\n1618\n[63.70]\nOVERALL LENGTH\n& ENGINE\n288.4\n[11.35]\n-\n[3.76]\n-\n63.5\n[2.50]\nB.C.\nØ 120.7\n[4.751\n167.1\n[6.58]\nIT\n-\n120.7\n[4.75]\n948\n[37.321\nu\nto\nVIEW IN DIRECTION\nOF ARROW "A"\n(XS9209 REMOVED)\n63.5\n[2.50) 184.2\n[7.25)\nVIEW IN DIRECTION\nOF ARROW "B"\n- 280.5\n(11.04]\nra\n+ 30\n[1.18\n& ENGINE\n476.2\n[18.75]\nRFOB\nFFOB\nTESNIWWO\n-\n-\n252.3\n[9.93]\n292.3., Ć CRANKSHAFT\n11 & CRANKSHAFT\n320.9\n[ 12.631\n345.9\n295.9\n[13.62]\n[11.65]\n[11.51] [CRANKSHAFT\nB\n& ENGINE\n-\n-\n-\n-\n355.5\n[14.00]\n337.5\n319.5 [13.29)\n[ 12.58]\n-- -+\n319.5\n355.5\n[12.58] [14.001\n337.5\n[13.29]\nCOD\na\n300.4\n-\n-\n325.4 [11.831\n350.4 (12.81)\nCOUPLING\nFLANGE [\nUL\n-\nHX9114\nB\n[13.791\nVB 9022 )\n69022\n-\nRG9237\n-\n200\

In [14]:
import json

RESULTS_DIR = 'jsons'

for idx, resp in enumerate(response.json()['document']):
    imgname = file_name
    jpath = join(RESULTS_DIR, basename(imgname) + '.json')
    with join(jpath, 'w') as f:
        datatxt = json.dumps(resp, indent = 2)
        print("Wrote", len(datatxt), "bytes to", jpath)
        f.write(datatxt)
        # print the plaintext to screen for convenience
        print("---------------------------------------------")
        t = resp['textAnnotations'][0]
        print("    Bounding Polygon:")
        print(t['boundingPoly'])
        print("    Text:")
        print(t['description']) 

AttributeError: 'AnnotateImageResponse' object has no attribute 'json'

In [5]:
#IN DEVELOPMENT - Google Vision API has beta capabilities to detect handwritten text too

def detect_handwritten_ocr_uri(uri):
    """Detects handwritten characters in the file located in Google Cloud
    Storage.

    Args:
    uri: The path to the file in Google Cloud Storage (gs://...)
    """
    from google.cloud import vision_v1p3beta1 as vision
    client = vision.ImageAnnotatorClient()
    image = vision.types.Image()
    image.source.image_uri = uri

    # Language hint codes for handwritten OCR:
    # en-t-i0-handwrit, mul-Latn-t-i0-handwrit
    # Note: Use only one language hint code per request for handwritten OCR.
    image_context = vision.types.ImageContext(
        language_hints=['en-t-i0-handwrit'])

    response = client.document_text_detection(image=image,
                                              image_context=image_context)

    print('Full Text: {}'.format(response.full_text_annotation.text))
    for page in response.full_text_annotation.pages:
        for block in page.blocks:
            print('\nBlock confidence: {}\n'.format(block.confidence))

            for paragraph in block.paragraphs:
                print('Paragraph confidence: {}'.format(
                    paragraph.confidence))

                for word in paragraph.words:
                    word_text = ''.join([
                        symbol.text for symbol in word.symbols
                    ])
                    print('Word text: {} (confidence: {})'.format(
                        word_text, word.confidence))

                    for symbol in word.symbols:
                        print('\tSymbol: {} (confidence: {})'.format(
                            symbol.text, symbol.confidence))

import os

file_name="textAnnotations.json"
#if statement below only required if writing to an existing file
#if os.path.exists(file_name):
   # with open(file_name, 'r') as f:
     #   all_data = json.load(f)
#else:

#Initiate a list for loading dict data
all_data = []

#Append the dict loaded with json data to variable

all_data.append(final_json['textAnnotations'])

#Write the all_data data to json file of file _name
with open(file_name, 'w') as f:
    json.dump(all_data, f)

print(textAnnotations)

In [48]:
file_name = 'Images/03_VDOT_Valvebox.jpg'
file_out = 'Output/03_VDOT_Valvebox_boxes.jpg'
render_doc_text(file_name, file_out)


Detected Text
5
Notes en bob 514131
REVITONCILIAT
REVISTONS
D ESCRIPTION
DATE
O
,
CHITTED ISIFIS HANDLE
Rih
FOR CLARITY
-3
100-52 FEMARS SNAP
RC-7 FIT
RII
15" TYP
/15. TYP
16xø
GEAR SHIFT HANDLE
SHIFT YOKE
DRIVE GEAR
RI
FINGER PIN (2) M
NO. 112
4X44
FE
APPROX. 13
MUST CLEAR
MINOR GEAR
TOOTH DIA
E
HL
n
sommige
GEAR SHIFT SUPPORT
PLATE DETAIL A
SCALE: 7/8:1
DRIVE SHAFT -
NO. 107
TRANSMISSION
HOUSING
NO. 109
FOLLOWER GEAR
NO. 111
COVER PLATE
NO. 108
SCSOLINE
DRIVE GEAR DETAIL*C
SCALE: FULL
RC-8 FIT
RC-7 FIT + 2x 5100-TRUAR SNAP
ne 8 RING 2 PLACES
2 PLACES L L
+ APPROX CLEARANCE
SHIFT YOKE DETAIL "B"
SCALE: 7/8:1
KNOB SPRING
Søh
GEAR SHIFT KNOB —
NO. 104
(NEUTRAL POSITION)
MEDIUM KNURL
GEAR SHIFT HANDLE
GASKET +
ex DEEP---
4X -20 X SOCKET HEAD CAP SPOTFACE
SCREW & MED LOCK WASHER
OR HI-COLLAR LOCK WASHER
e
me POSITION
NOTE: BOTTOM TAPPED
HOLES SHOWN IN THIS
- 2x
DRIVE GEAR
NO. 106
SEE DETAIL "C)
.c.
B1212 TORRINGTON-
NEEDLE BEARING
DOHEL PIN
SHIFT YOKE
SEE DETAIL *8')
DOWEL PIN
GEAR SHIFT 

In [31]:
print(response.full_text_annotation)

pages {
  property {
    detected_languages {
      language_code: "en"
      confidence: 0.8600000143051147
    }
    detected_languages {
      language_code: "sl"
      confidence: 0.009999999776482582
    }
    detected_languages {
      language_code: "si"
      confidence: 0.009999999776482582
    }
    detected_languages {
      language_code: "ca"
      confidence: 0.009999999776482582
    }
    detected_languages {
      language_code: "fi"
      confidence: 0.009999999776482582
    }
    detected_languages {
      language_code: "haw"
      confidence: 0.009999999776482582
    }
  }
  width: 1797
  height: 1381
  blocks {
    bounding_box {
      vertices {
        x: 1681
        y: 25
      }
      vertices {
        x: 1748
        y: 25
      }
      vertices {
        x: 1748
        y: 46
      }
      vertices {
        x: 1681
        y: 46
      }
    }
    paragraphs {
      bounding_box {
        vertices {
          x: 1681
          y: 25
        }
        vertic

In [35]:
print(document.text)

18/80
2
3 3
3 3 O
E
P
ICECRETO
zasluzernd
@@@
POTEZEECE GREECETECTETIT
1000®®®®®®®®
ܠܠܠܠܠܠܠܠ
@ @
NIZ
10
@
AAN
® ®
W WINNI
®
O
“මල්ලී
ම
ම
ම ම ම ම ම ම ම ම ම ම
OBOOOOOOO
ITEM QUAN PART NO. DESCRIPTION
The ANSMISSION MOUNTING PAD
18033-D TRANSMISSION CASE
5766-18 x 144 LONG Soc. HD. BOLT
INSIDE SHIFT KNOB
18028-A INSIDE SHIFT LEVER
SHIFTING PIVOT PIN
15243- A RETAWING WASHER
TRU. 5100-75 SNAP RING
SHIFTING SHAFT
GS-2218-A SHIFT KNOB
180GI-A SHIFT LEVER
FRAME
F-319-A BUSNING
S-226-A AIR FILTER
18/72-A OIL DIP STICK
18124. A SHIFTING SHAFT CLAMP WASHER
18123-A PULLEY CLAMP WASHER
18/71-A PULLEY SPACER
N.D. Z 9507 BALL BRG
TRU. 5000-28/SNAP RING
18003. A BEARING SPACER
18178-B BEARING HOUSING
BENING HOUSING GASKET
18091-A TRANSMISSION COVER GASKET
18/10-8 TRANSMISSION COVER
ISSANN
ITEMIQUA PART No. DESCRIPTION
14-20 * H LONG ROUND HE SCREW
14' PIPE PLUG
15045-A CLAMP WASHER
N.D. 43201 BALL B'R's.
18022-A BEARING COVER
18089-A BEARING COVER GASKET
18004- GEAR SPACER
18014. A 58 TGEAR
18015. A 

In [37]:
print(document.json)

AttributeError: 'TextAnnotation' object has no attribute 'json'

In [38]:
print(enumerate(document))

TypeError: 'TextAnnotation' object is not iterable